# TPPI: Talking Point Performance Index

**Alejandro Medrano San Clemente**

In [6]:
class TPPIResult(TypedDict):
    TPPI: float
    BertScore: float
    Normalized_BertScore: float
    Perplexity: float
    Normalized_Perplexity: float
    Flesch: float
    Normalized_Flesch: float


class TPPI:
    def __init__(self, model_type: str = 'bert-base-uncased'):
        """
        Inicializa el normalizador de puntuaciones y los modelos necesarios para los cálculos.
        """
        self.scorer = BERTScorer(model_type=model_type)
        self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

    @staticmethod
    def normalize_bert_score(bert_score: float) -> float:
        """
        Normaliza el puntaje BERT en un rango de 0 a 1.
        """
        return (bert_score + 1) / 2

    @staticmethod
    def normalize_perplexity(perplexity: float) -> float:
        """
        Normaliza el valor de la perplejidad en un rango de 0 a 1.
        """
        # Aseguramos que la perplexidad mínima sea al menos 1 para evitar división por cero
        perplexity = max(perplexity, 1)
        # Invertimos la fórmula para que valores bajos de perplexidad den valores altos normalizados
        return 1 - (min(perplexity, 100) - 1) / 99

    @staticmethod
    def normalize_flesch(flesch: float) -> float:
        """
        Normaliza el puntaje de Flesch en un rango de 0 a 1.
        """
        return max(min(flesch, 100), 0) / 100

    def calculate_bert_score(self, reference_text: str, generated_text: str) -> float:
        """
        Calcula el BertScore entre un texto de referencia y uno generado.
        """
        P, R, F1 = self.scorer.score([reference_text], [generated_text], verbose=False)
        bert_score = F1.item()
        return bert_score

    def calculate_perplexity(self, text: str) -> float:
        """
        Calcula la perplejidad de un texto utilizando el modelo GPT-2.
        """
        encode = self.gpt2_tokenizer.encode(text, return_tensors='pt')
        with torch.no_grad():
            loss = self.gpt2_model(encode, labels=encode)[0]
        perplexity = torch.exp(loss).item()
        return perplexity

    @staticmethod
    def calculate_flesch(text: str) -> float:
        """
        Calcula el puntaje de Flesch Reading Ease de un texto.
        """
        flesch_score = flesch_reading_ease(text)
        return flesch_score

    def calculate_tppi(self, original_text: str, generated_text: str) -> TPPIResult:
        """
        Calcula el TPPI basado en los textos de referencia y generados, devolviendo los puntajes de BERT,
        perplejidad y Flesch tanto normalizados como sin normalizar.
        """
        # Calcula BertScore
        bert_score = self.calculate_bert_score(original_text, generated_text)
        normalized_bert_score = self.normalize_bert_score(bert_score)

        # Calcula Perplejidad
        perplexity = self.calculate_perplexity(generated_text)
        normalized_perplexity = self.normalize_perplexity(perplexity)

        # Calcula Flesch
        flesch = self.calculate_flesch(generated_text)
        normalized_flesch = self.normalize_flesch(flesch)

        # Calcula TPPI
        tppi_score = 0.5 * normalized_bert_score + 0.25 * normalized_perplexity + 0.25 * normalized_flesch

        # Devuelve todos los valores en un TypedDict
        return TPPIResult(
            TPPI=tppi_score,
            BertScore=bert_score,
            Normalized_BertScore=normalized_bert_score,
            Perplexity=perplexity,
            Normalized_Perplexity=normalized_perplexity,
            Flesch=flesch,
            Normalized_Flesch=normalized_flesch
        )


In [3]:
tppi_calculator = TPPI()

/Users/alejandro.medrano/.local/share/virtualenvs/UNITE_TALKING_POINTS-QWuKLflF/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
tppi_calculator.calculate_tppi("the courtyard of my house is particular", "the courtyard of my house sometimes is particular")

{'TPPI': 0.8862211563825607,
 'BertScore': 0.9112846255302429,
 'Normalized_BertScore': 0.9556423127651215,
 'Perplexity': 1019.5551147460938,
 'Normalized_Perplexity': 1.0,
 'Flesch': 63.36,
 'Normalized_Flesch': 0.6335999999999999}

In [5]:
tppi_calculator.calculate_tppi(
    "the courtyard of my house is particular",
    "The essence of the random text platypus, which has long been synonymous with Skype, is difficult to swimming pool or improve."
)

{'TPPI': 0.7683239503622055,
 'BertScore': 0.402495801448822,
 'Normalized_BertScore': 0.701247900724411,
 'Perplexity': 164.62295532226562,
 'Normalized_Perplexity': 1.0,
 'Flesch': 67.08,
 'Normalized_Flesch': 0.6708}